LIBRARIES

In [181]:
import pandas as pd
import json
import string
import unidecode
import openai
import sys
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.corpus import stopwords
from matplotlib import pyplot as plt
from sklearn.decomposition import LatentDirichletAllocation, NMF
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.feature_extraction.text import TfidfVectorizer

PARAMETROS

In [182]:
#parametros de entrada
# qty_topics
#lista_stopwords
#filtros de la query

try:
    qty_topics = sys.argv[1]
except:
    pass

try:
    aux_stop_dashboard = sys.argv[2]
except:
    pass

try:
    col_id = sys.argv[3]
except:
    pass


#PARAMETROS DE ENTRADA
file_name='train1.csv' #en verdad es la QUERY


aux_stop_dashboard=["lista parametro de entrada"] #la lista de stopwords temporales del dashboard
qty_topics=6 
col_id='Unnamed: 0' #La columna de la query de los ids de las interacciones

In [184]:
#PARAMETROS INTERNOS
decoder=False
qty_words=7 # cantidad de palabras por topico

LOAD & STOPWORDS

In [246]:
df=pd.read_csv(file_name)

In [247]:
# with open("stop_words_spanish.txt", "rt") as f:
    # a=f.readlines()

stop = stopwords.words("spanish")
# a = [w[:-1] for w in a]
aux_stop=['aver','ahi','tarjeta','tarjetas','nombre','marido','osvaldo','hermano','vos','ah','hola','que','tal','santander','verdad','eh',
    'si','queria','hacer','buen','consulta','bueno','bien','dia','dale','ok','ah','eh', 'favor','mira',
    'segundo','segundito','escucha', 'ahora','escucho','dale','perfecto','oh' ,'ayudar','gracias','muchisimas',
'igualmente','entonces','bueno','amigo','digamos','usted','si','ver','dias','hola','escuchas','perdon','no'
'tal','hablando','tardes','noches','buen','eh','bien','acá','dia','pues','ayer','como','luego','va','creo',
'despues','señor','señora','digame','viste','hace','eh','sabes','hijo','noviembre','diciembre','octubre','mes',
'hola','vez','dos','mercadolibre','mismo','elba']
stop.extend(aux_stop_dashboard)
stop.extend(aux_stop)
stop.extend(['asi','nan','mas',"quiero", "se", "hola", "ya", "dia", "buen", "tardes", "gracias", "problema", "te", "quería", "eh", "vos", "mí", "perfecto","listo","dale","ay", "ah", "claro", "tenes", "tu", "mira", "aca","vas", "vale", "pasa", "ok", "mis","veo","favor", "señor", "pido", "digamos", "digo", "viste", "senora", "senor",'elba'])

PREPROCESSING

In [248]:
df["text"].astype(str)
texts = df["text"].values
translator = str.maketrans(string.punctuation, ' '*len(string.punctuation))

texts = [str(s).translate(translator) for s in texts] #Ajusta un poco las palabras
texts = [word_tokenize(s) for s in texts] #tokeniza
texts = [[w.lower() for w in s] for s in texts] #a lower
texts = [[unidecode.unidecode(w) for w in s] for s in texts] #Elimina acentos
texts = [['' if t in stop else t for t in s] for s in texts] #elimina las palabras que estan en una lista
texts = [" ".join(s) for s in texts]
df['text']=texts
# vec = TfidfVectorizer(min_df = 10, max_df = 1.0, ngram_range=(1,3), stop_words= a) 
# X = vec.fit_transform(texts)

MODELS

In [249]:
# create dataset

df_filt=df.copy()

tfidf = TfidfVectorizer(ngram_range=(1,2)) #(min_df = 10, max_df = 1.0, ngram_range=(1,2), stop_words= stop) 
text_sutopic=df_filt['text']
df_text_sutopic=text_sutopic.reset_index()

vectorizer = tfidf.fit_transform(text_sutopic)
lda = NMF(n_components=qty_topics,random_state=0,max_iter=1000)
lda.fit(vectorizer)

NMF(max_iter=1000, n_components=6, random_state=0)

In [250]:
lda.transform(vectorizer)[0][2]
# df_filt.loc[r,'topic '+str(j+1)]

0.0456258068119757

In [251]:
dict_topics_per_int={}
aux=0
for r in df_filt[col_id]:
    dict_topic={}
    for j in range(0,qty_topics):
        # df_filt['topic '+str(j+1)]=-1.0
        df_filt.loc[r,'topic '+str(j+1)]=lda.transform(vectorizer)[aux][j]
        dict_topic['topic '+str(j+1)]=lda.transform(vectorizer)[aux][j]
    dict_topics_per_int[str(r)]=dict_topic
    aux=aux+1

In [190]:
dict_top_interactions={}
dict_list_words=[]
dict_top_interactions={}
dict_list_inter=[]
dic_topic={}

num_of_topic=0
pedido = "Resume el siguiente texto en menos de 15 palabras: "

for topic_idx, topic in enumerate(lda.components_):
        num_of_topic=num_of_topic+1
        top_features_ind = topic.argsort()[: - qty_words-1 : -1]
        top_features = [ tfidf.get_feature_names_out()[r] for r in top_features_ind]
        dict_top_words={}
        for j in range(qty_words):        
                dict_top_words[str(j+1)] = top_features[j]
        dict_list_words.append(dict_top_words)
        topics_list = lda.transform(vectorizer)
        topcis_in_documents=pd.DataFrame(topics_list)
        index_topics_more_representatives=topcis_in_documents[num_of_topic-1].nlargest(qty_interactions).index
        dict_top_interactions={}
        # for i in range(1):
                # dict_top_interactions[str(i+1)]=df_text_sutopic.loc[index_topics_more_representatives[i],'text']
        list_interaction=df_text_sutopic.loc[index_topics_more_representatives[0],'text']
        
        # FOR THE SUMMARY
        if decoder==True:
                response = openai.Completion.create(
                model="text-davinci-003",
                prompt= pedido + list_interaction,
                temperature=0.1,
                max_tokens=128,
                top_p=1.0,
                frequency_penalty=0.0,
                presence_penalty=0.0
                )
                texto = response['choices'][0]['text']
                
                dict_list_inter.append(texto)
        else:
                dict_list_inter.append(list_interaction)

In [191]:
dict_topic={}
for t in range(len(dict_list_words)):
    dict_topic['topic '+str(t+1)]={'words':dict_list_words[t],'interaction':dict_list_inter[t]}

In [192]:
with open('word_interactions_topics.json', 'w') as f:
    json.dump(dict_topic, f, indent=4)
    json.dump(dict_topics_per_int, f, indent=4)

In [253]:
df_filt.to_csv('topics_for_interractions.csv')